# <font color='red'> IBM Data Science Capstone by Coursera </font> 

## **Week 5 Final Report**

### Toronto, Ontario - A shopping mall venture

##### steps undertaken 

* Researching and creating a database containing various neighbourhoods using webscraping 
* Getting the geographical coordinates of the said neighborhoods
* Obtain the venue data using the FourSquare API
* Exploring and clustering neighborhoods
* Selecting the best cluster for shopping mall developmenent 

In [1]:
#importing necessary libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
#from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [117]:
from bs4 import BeautifulSoup # library to parse HTML and XML documents


In [118]:
import csv

In [114]:
pd.set_option('max_colwidth', 800)

## instantiating BeautifulSoup object and reading the table from wikipedia page

In [256]:
# we use the get method to retrive the data off the wikipedia page and use the lxml parser to scrape the data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

### Creating a csv_writer to append the scraped content in an xlx file by initally defining column names 

In [257]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

### Main Scraping of data begins here

In [258]:
table = soup.find('table', class_ = 'wikitable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)


### closing the CSV file

In [259]:
csv_file.close()

### Creating a pandas dataframe

In [260]:
toronto_df=pd.read_csv('toronto_postal_codes.csv')

### Gauging the shape of the created pandas dataframe

In [261]:
toronto_df.shape

(180, 3)

### Final Reformatting and resultant dataframe

In [262]:
toronto_df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A\r\n,Not assigned\r\n,NaN
1,M2A\r\n,Not assigned\r\n,NaN
2,M3A\r\n,North York\r\n,Parkwoods
3,M4A\r\n,North York\r\n,Victoria Village
4,M5A\r\n,Downtown Toronto\r\n,Regent Park / Harbourfront


In [263]:
toronto_df=toronto_df.replace('\r\n', '',regex=True)

In [264]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [265]:
toronto_df_copy=toronto_df #making a copy for redundancy 

In [266]:
drop_index=toronto_df.loc[toronto_df['Borough']=='Not assigned'].index

In [267]:
drop_index

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [268]:
toronto_df_copy.drop(drop_index,inplace=True)

In [269]:
toronto_df_copy

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [270]:
toronto_df_copy.reset_index(drop=True,inplace=True)

### we replace the / in the neighbourhood column with , for easy reading 

In [271]:
toronto_df_copy['Neighbourhood'].replace('/',',',regex=True,inplace=True)

In [272]:
toronto_df_copy.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## Getting coordinates for the different boroughs

In [226]:
# define a function to get coordinates
def get_coordinates(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Canada'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [227]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [get_coordinates(neighborhood) for neighborhood in toronto_df_copy['Neighbourhood'].tolist()]

In [228]:
coords

[[43.68657452246789, -79.40999250225033],
 [43.73154000000005, -79.31427999999994],
 [43.65974052181429, -79.3615640533021],
 [43.72357000000005, -79.43710999999996],
 [43.666630269551845, -79.39326783273027],
 [43.66321327878551, -79.53150965640926],
 [43.81023000000005, -79.22037999999998],
 [43.705680011617886, -79.33385830530915],
 [43.70626000000004, -79.30090999999999],
 [43.65794000000005, -79.37561999999997],
 [43.70842942383775, -79.44648154184813],
 [43.65297000000004, -79.55741999999998],
 [43.789480000000026, -79.17613999999998],
 [43.705680011617886, -79.33385830530915],
 [43.68752000000006, -79.32058999999998],
 [43.67110000000008, -79.37358999999998],
 [43.68942000000004, -79.42697999999996],
 [43.63349000000005, -79.57073999999994],
 [43.76602147574681, -79.18538783940997],
 [43.67413000000005, -79.29643999999996],
 [43.64829993392594, -79.37524996335624],
 [43.68857000000003, -79.45482999999996],
 [43.767480000000035, -79.22828999999996],
 [43.70023937855272, -79.35106

In [229]:
## creating a dataframe to store the retreived coordinates
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [236]:
toronto_df_copy.reset_index(drop=True,inplace=True)

In [237]:
toronto_df_copy['Latitude']=df_coords['Latitude']
toronto_df_copy['Longitude']=df_coords['Longitude']

In [274]:
loc_df=pd.read_csv('Geospatial_Coordinates.csv')

In [275]:
loc_df.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [276]:
toronto_df_copy=pd.merge(toronto_df_copy,loc_df, on='Postcode', how='inner')

In [277]:
toronto_df_copy

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [278]:
toronto_df_copy.shape

(103, 5)

In [279]:
# save the DataFrame as CSV file
toronto_df_copy.to_csv("toronto.csv", index=False)

### Using the Folium library to superimpose the boroughs of Toronto on the map

In [240]:
# get the coordinates of Toronto, Canada
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada is  {}, {}.'.format(latitude, longitude))

GeocoderServiceError: [Errno 11001] getaddrinfo failed

In [280]:
toronto_map=folium.Map(location=[latitude,longitude],zoom_start=10)

#adding markers to the map
for lat,lng,borough,neigh in zip(toronto_df_copy['Latitude'],toronto_df_copy['Longitude'],toronto_df_copy['Borough'],toronto_df_copy['Neighbourhood']):
    label='{},{}'.format(neigh,borough) #labels would prompt borough and neighbourhood name
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lat, lng],   #circle markers for every plotted latitude and longitude
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map).add_to(toronto_map)
    
toronto_map

In [281]:
# save the map as HTML file
map_toronto.save('map_toronto.html')


### Using the FourSquare API to get the venue data

In [282]:
CLIENT_ID='JAILAMIF4H0FPZZ22ZKU4LDPQJBRNE4N1C3YH4NYH5OZNBL1' # Foursquare ID
CLIENT_SECRET='BP4XAUNU52R3G1LEWLKWOPLSTHCLMTOWCR2YW05LZIPQGOPM' # Foursquare Secret
VERSION='20180605'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JAILAMIF4H0FPZZ22ZKU4LDPQJBRNE4N1C3YH4NYH5OZNBL1
CLIENT_SECRET:BP4XAUNU52R3G1LEWLKWOPLSTHCLMTOWCR2YW05LZIPQGOPM


## We reduce the Foursquare API calls by reducing the results only to include boroughs named "toronto"

In [243]:
toronto_borough = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_df_copy = toronto_df_copy[toronto_df_copy['Borough'].isin(toronto_borough)].reset_index(drop=True)
print(toronto_df_copy.shape)
toronto_df_copy.head()

(39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.659741,-79.361564
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.666630,-79.393268
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657940,-79.375620
3,M5C,Downtown Toronto,St. James Town,43.671100,-79.373590
4,M4E,East Toronto,The Beaches,43.674130,-79.296440


In [255]:
toronto_df_copy.shape

(39, 5)

### implementing the calls to API to get the top 100 venues in the neighborhood withing 500m vicinity

In [289]:
radius = 500
LIMIT = 100

venues_list = []

for lat,lng, post, borough, neighborhood in zip(toronto_df_copy['Latitude'],toronto_df_copy['Longitude'],toronto_df_copy['Postcode'],toronto_df_copy['Borough'],toronto_df_copy['Neighbourhood']):
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items'] # the necessary data for consideration, venue details lies within the items sublist, within groups sublist which is inturn within the "response" master list
    
    for v in results:
        venues_list.append((
                post,
                borough,
                neighborhood,
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']))

In [286]:
toronto_df_venues=pd.DataFrame(venues_list)

In [291]:
toronto_df_venues.columns = ['Postcode', 
                     'Borough', 
                     'Neighbourhood',
                     'Borough Latitude', 
                     'Borough Longitude', 
                     'Venue Name',
                     'Venue Latitude',
                     'Venue Logitude',
                     'Venue Category']
print(toronto_df_venues.shape)


(2128, 9)


In [292]:
toronto_df_venues.groupby('Borough').count()

,Postcode,Neighbourhood,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Logitude,Venue Category
Borough,,,,,,,,
Central Toronto,111,111,111,111,111,111,111,111
Downtown Toronto,1227,1227,1227,1227,1227,1227,1227,1227
East Toronto,123,123,123,123,123,123,123,123
East York,77,77,77,77,77,77,77,77
Etobicoke,76,76,76,76,76,76,76,76
Mississauga,12,12,12,12,12,12,12,12
North York,244,244,244,244,244,244,244,244
Scarborough,91,91,91,91,91,91,91,91
West Toronto,151,151,151,151,151,151,151,151


In [293]:
toronto_df_venues.groupby('Neighbourhood').count()

,Postcode,Borough,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Logitude,Venue Category
Neighbourhood,,,,,,,,
Agincourt,5,5,5,5,5,5,5,5
"Alderwood , Long Branch",10,10,10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",25,25,25,25,25,25,25,25
Berczy Park,57,57,57,57,57,57,57,57
"Birch Cliff , Cliffside West",4,4,4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18,18,18


#### **Lets get the number of unique categories in the venues**

In [296]:
print('There are {} uniques categories.'.format(len(toronto_df_venues['Venue Category'].unique())))

There are 268 uniques categories.


#### printing the first 50 of the unique venues list


In [297]:
# print out the list of categories
toronto_df_venues['Venue Category'].unique()[:50]

array(['Park', 'Food & Drink Shop', 'Construction & Landscaping',
       'Hockey Arena', 'Coffee Shop', 'Portuguese Restaurant',
       'French Restaurant', 'Intersection', 'Pizza Place', 'Bakery',
       'Breakfast Spot', 'Distribution Center', 'Spa', 'Restaurant',
       'Gym / Fitness Center', 'Historic Site', 'Farmers Market',
       'Chocolate Shop', 'Pub', 'Performing Arts Venue', 'Dessert Shop',
       'Mexican Restaurant', 'Yoga Studio', 'Café', 'Theater',
       'Event Space', 'Ice Cream Shop', 'Shoe Store', 'Art Gallery',
       'Asian Restaurant', 'Cosmetics Shop', 'Bank', 'Electronics Store',
       'Beer Store', 'Hotel', 'Health Food Store', 'Wine Shop',
       'Antique Shop', 'Boutique', 'Furniture / Home Store',
       'Vietnamese Restaurant', 'Clothing Store', 'Accessories Store',
       "Women's Store", 'Gift Shop', 'Miscellaneous Shop',
       'Sushi Restaurant', 'Italian Restaurant', 'Creperie', 'Beer Bar'],
      dtype=object)

In [298]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in toronto_df_venues['Venue Category'].unique()

True

### We one hot encode different categories to better analyse boroughs

In [311]:
toronto_df_onehot=pd.get_dummies(toronto_df_venues[['Venue Category']],prefix="",prefix_sep="")

In [312]:
toronto_df_onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### as we can see there is no "Borough column" to provide insight about so we add it back from the toronto_df_venue df

In [313]:
toronto_df_onehot['Neighbourhood']=toronto_df_venues['Neighbourhood']
toronto_df_onehot.head()
#move Borough to first column
fixed_columns = [toronto_df_onehot.columns[-1]] + list(toronto_df_onehot.columns[:-1])
toronto_df_onehot = toronto_df_onehot[fixed_columns]
toronto_df_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [314]:
toronto_df_onehot.shape

(2128, 269)

### We now group boroughs by venue categories factoring in the mean of frequency of the same 

In [315]:
toronto_df_group=toronto_df_onehot.groupby('Neighbourhood').mean().reset_index()

In [316]:
toronto_df_group.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [317]:
toronto_df_group.shape

(95, 269)

In [318]:
len(toronto_df_group[toronto_df_group["Shopping Mall"] > 0])

12

### Creating a new dataframe for mall data 

In [319]:
toronto_mall=toronto_df_group[["Neighbourhood","Shopping Mall"]]

In [320]:
toronto_mall

,Neighbourhood,Shopping Mall
0,Agincourt,0.000000
1,"Alderwood , Long Branch",0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.050000
3,Bayview Village,0.000000
4,"Bedford Park , Lawrence Manor East",0.000000
5,Berczy Park,0.017544
6,"Birch Cliff , Cliffside West",0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.000000
8,Business reply mail Processing CentrE,0.000000
9,"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",0.000000


### Running K-means Algorithm to facilitate clustering

In [322]:
# set number of clusters
nclusters = 3

toronto_clustering = toronto_mall.drop(["Neighbourhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=nclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0])

In [323]:
# create a new dataframe that includes the cluster as well as the shooping mall clusters for each neighborhood.
toronto_merged = toronto_mall.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

In [324]:
toronto_merged

,Neighbourhood,Shopping Mall,Cluster Labels
0,Agincourt,0.000000,0
1,"Alderwood , Long Branch",0.000000,0
2,"Bathurst Manor , Wilson Heights , Downsview North",0.050000,2
3,Bayview Village,0.000000,0
4,"Bedford Park , Lawrence Manor East",0.000000,0
5,Berczy Park,0.017544,0
6,"Birch Cliff , Cliffside West",0.000000,0
7,"Brockton , Parkdale Village , Exhibition Place",0.000000,0
8,Business reply mail Processing CentrE,0.000000,0
9,"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",0.000000,0


In [325]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_df_copy.set_index("Neighbourhood"), on="Neighbourhood")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(100, 7)


,Neighbourhood,Shopping Mall,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,Agincourt,0.00,0,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood , Long Branch",0.00,0,M8W,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor , Wilson Heights , Downsview North",0.05,2,M3H,North York,43.754328,-79.442259
3,Bayview Village,0.00,0,M2K,North York,43.786947,-79.385975
4,"Bedford Park , Lawrence Manor East",0.00,0,M5M,North York,43.733283,-79.419750


### Visualizing the clusters formed 

In [329]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(nclusters)
ys = [i+x+(i*x)**2 for i in range(nclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [330]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examining the clusters

### Cluster 0

In [331]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Neighbourhood,Shopping Mall,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,Agincourt,0.000000,0,M1S,Scarborough,43.794200,-79.262029
1,"Alderwood , Long Branch",0.000000,0,M8W,Etobicoke,43.602414,-79.543484
3,Bayview Village,0.000000,0,M2K,North York,43.786947,-79.385975
4,"Bedford Park , Lawrence Manor East",0.000000,0,M5M,North York,43.733283,-79.419750
5,Berczy Park,0.017544,0,M5E,Downtown Toronto,43.644771,-79.373306
6,"Birch Cliff , Cliffside West",0.000000,0,M1N,Scarborough,43.692657,-79.264848
7,"Brockton , Parkdale Village , Exhibition Place",0.000000,0,M6K,West Toronto,43.636847,-79.428191
8,Business reply mail Processing CentrE,0.000000,0,M7Y,East Toronto,43.662744,-79.321558
9,"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",0.000000,0,M5V,Downtown Toronto,43.628947,-79.394420
10,Caledonia-Fairbanks,0.000000,0,M6E,York,43.689026,-79.453512


### Cluster 1

In [332]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Neighbourhood,Shopping Mall,Cluster Labels,Postcode,Borough,Latitude,Longitude
89,"Wexford , Maryvale",0.166667,1,M1R,Scarborough,43.750072,-79.295849


### Cluster 2

In [333]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Neighbourhood,Shopping Mall,Cluster Labels,Postcode,Borough,Latitude,Longitude
2,"Bathurst Manor , Wilson Heights , Downsview North",0.050000,2,M3H,North York,43.754328,-79.442259
16,"Clarks Corners , Tam O'Shanter , Sullivan",0.066667,2,M1T,Scarborough,43.781638,-79.304302
24,Downsview,0.058824,2,M3K,North York,43.737473,-79.464763
24,Downsview,0.058824,2,M3L,North York,43.739015,-79.506944
24,Downsview,0.058824,2,M3M,North York,43.728496,-79.495697
24,Downsview,0.058824,2,M3N,North York,43.761631,-79.520999
47,Leaside,0.030303,2,M4G,East York,43.709060,-79.363452


# Observations

As is evidently visible the cluster 0 has a paucity in the sector of shopping malls and complexes and there is slight to almost negligible competition from the existing shopping mall complexes, and this presents a wonderful opportunity for construction of new shopping complexes, furthermore with areas such as Scarborough,with **population being 632,000** and **median income being around $59,000 **, Etibicoke, with  population being ** 345,000 ** and ** median income being $70,500 ** and the areas of York University** would definitely be boonful for upcoming mercantile businesses , following up is the cluster 1, which has a moderate presence of competitor shopping malls, the area of **Wexford and Maryvale** in Scarborough borough, estimated 30% of the population in Wexford being categorized as young and with a population of around 28,000 people and the maximum cohort earnign in the tune of about $35,000 to $52,000, it is certainly going to be challanging for a new mall venture to be established here unless the developers and retail strategists have a unique USP to stand out in the competition.
Lastly we have the extremely competitive market of cluster 2 with the inclusive boroughs and neighborhoods, being among the more financial successful ones in Toronto, of particular notice is again the borough of north york and East york, the neighborhoods of Downsview, having a population of around 36,000 people and despite being a medium earning neighborhood is sprawling with low to mid size shopping complexes, with the borough of North York with the average household income of around $82,000.
Therfore from the cluster analysis above, it becomes an imperative for any retail strategiest or property developer to target the cluster 0, and more specifically the neighborhoods in the areas of Scarborough and Etibicoke and Downtown, as they house greater population as well as a higher income earning populace too, and with slim competition in the area, it becomes an even more lucrative investment area,
Cluster 1, with its medium competitive presence may pose certain challanges in the establishment of a new shopping mall venture, and while certainly not impossible, the retail strategists and the property developers alike might have to come up with USPs plausible enough to stand out from the crowd, and the NO GO zone becomes cluster 2 as it is already rife with competition in the area, and with such well established comeptitors catering the neighborhood, new establishment here is highly unadvised.